<a href="https://colab.research.google.com/github/AdaobiNwafor/Coursera_Capstone_/blob/main/Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This notebook will be used for the capstone project in the IBM Data Science Professional Certificate cCoursera.**

In [2]:
import pandas as pd
import numpy as np

In [8]:
print('hello capstone project course!')

hello capstone project course!
